## 4.1 (Felder)

Um die aussagekräftigsten Felder auszuwählen, habe ich darauf geachtet, welche Felder überhaupt einen realistischen Einfluss 
auf den Preis haben könnte wie im Verlauf im echten Leben. In meinem Fall sind Felder wie "year", "manufacturer", "model" aber auch "condition" in Frage gekommen.

**year:** Neuwertige Autos sind teurer -> direkt proportional zum Preis.

**manufacturer:** Hersteller wie BMW oder Mercedes sind meist teurer als Kia oder Ford. 

**model:** Manche Modelle sind teurer (z.b. BMW 420 zu BMW 430), aber viele Ausreisser(ACHTUNG).

**condition:** Ein Auto in einem nicht so gutem Zustand wird meist günstigerer gehandelt als das dem Normal-Zustand.

**type:** SUV's oder Pickup's kosten meist mehr als Kleinwagen.

**transmission:** Automatik kann teurer sein, ist aber nicht überall gleich.

**fuel:** Elektroautos oder Dieselmodelle haben oft andere Preisstrukturen.

------------------------------------------------------------------------------------------------------------------

## 4.2 (MSE Berechnen)

## Vorteile von MSE:
- Betont grosse Fehler stärker weil Fehler quadriert werden (Ausreisser).
- Nützlich, wenn du grosse Abweichungen besonders vermeiden willst.

In [21]:
import pandas as pd

df = pd.read_csv(r"C:\Users\AMAR\OneDrive - BBBaden\vehiclesLB259.csv", sep=';')

df.head(9998)

,region,price,year,manufacturer,model,condition,cylinders,fuel,transmission,type,paint_color,state
0,auburn,33590,2014,gmc,sierra 1500 crew cab slt,good,8 cylinders,gas,other,pickup,white,al
1,auburn,22590,2010,chevrolet,silverado 1500,good,8 cylinders,gas,other,pickup,blue,al
2,auburn,39590,2020,chevrolet,silverado 1500 crew,good,8 cylinders,gas,other,pickup,red,al
3,auburn,30990,2017,toyota,tundra double cab sr,good,8 cylinders,gas,other,pickup,red,al
4,auburn,15000,2013,ford,f-150 xlt,excellent,6 cylinders,gas,automatic,truck,black,al
...,...,...,...,...,...,...,...,...,...,...,...,...
9993,los angeles,32997,2018,bmw,540i bad credit,like new,6 cylinders,gas,automatic,sedan,black,ca
9994,los angeles,4500,1956,ford,thunderbird,fair,8 cylinders,gas,automatic,convertible,red,ca
9995,los angeles,295,2013,honda,accord lx,like new,4 cylinders,gas,automatic,sedan,silver,ca
9996,los angeles,15777,2014,dodge,charger se,like new,6 cylinders,gas,automatic,sedan,white,ca


In [33]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

df = pd.read_csv(r"C:\Users\AMAR\OneDrive - BBBaden\vehiclesLB259.csv", sep=';')

# Ausreißer entfernen
df = df[df['price'] < 100000]

features = ['year', 'manufacturer', 'fuel', 'transmission', 'type', 'paint_color', 'state']
target = 'price'

X = df[features]
y = df[target]

categorical = ['manufacturer', 'fuel', 'transmission', 'type', 'paint_color', 'state']
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical)
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('model', LinearRegression())
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

# Ausgabe
print("MSE:", round(mse, 2))


MSE: 114558531.22


-------------------------------------------------------------------------------------------------------------------------

## 4.3 (Wahrheitsmatrix)

In [49]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# Wahrheitsmatrix
conf_matrix = confusion_matrix(y_test, y_pred)

TN, FP, FN, TP = conf_matrix.ravel()

print(f"{TP} → Preis war teuer, wurde korrekt erkannt (= True Positive)")
print(f"{FP} → Preis war nicht teuer, aber als teuer vorhergesagt (= False Positive)")
print(f"{FN} → Preis war teuer, aber als nicht teuer vorhergesagt (= False Negative)")
print(f"{TN} → Preis war nicht teuer, wurde korrekt erkannt (= True Negative)")

# Precision und Recall berechnen
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print(f"\nPrecision: {precision:.2f}")
print(f"Recall: {recall:.2f}")


772 → Preis war teuer, wurde korrekt erkannt (= True Positive)
178 → Preis war nicht teuer, aber als teuer vorhergesagt (= False Positive)
246 → Preis war teuer, aber als nicht teuer vorhergesagt (= False Negative)
804 → Preis war nicht teuer, wurde korrekt erkannt (= True Negative)

Precision: 0.81
Recall: 0.76


---------------------------------------------------------------------------------------------------

## 4.4 (Gut oder Schlecht?)

Mein Modell funktioniert insgesamt ziemlich gut: Mit einer Präzision von 0.81 und einem Abruf von 0.76 
erkennt es teure Autos zuverlässig, macht aber auch ein paar Fehler.Gut klappt es bei bekannten Herstellern und neueren Modellen. Obwohl die Zahl doch noch eher nicht so einen grossen Unterschied macht. Ich vermutem, dass die Genauigkeit noch besser wäre, wenn man z.B. Modellnamen vereinfacht oder irrelevante Ausreisser entfernt. Viellebeeinflussen auch unsaubere Angaben (z.B. bei "condition" oder "cylinders") die Vorhersagen negativ. Insgesamt bin ich mit dem Ergebnis zufrieden, könnte aber noch besser sein, vor allem 0 Werte entfernen. 